In [1]:

from monai.utils import first, set_determinism
from monai.transforms import (EnsureChannelFirstd, Compose, CropForegroundd, LoadImaged, Orientationd, RandCropByPosNegLabeld, ScaleIntensityRanged, Spacingd)
from monai.networks.nets import DynUNet
from monai.networks.layers import Norm
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset
from monai.apps import download_and_extract
from monai.transforms import CenterSpatialCropd
from monai.transforms import Resized
import torch
import matplotlib.pyplot as plt
import os
import glob
import torch.nn as nn
import json
from datetime import datetime
import numpy as np
import nibabel as nib

import math
import os
import glob
import torch
import numpy as np
import optuna
from optuna.integration import PyTorchLightningPruningCallback
from functools import partial
from monai.networks.nets import DynUNet
from monai.data import CacheDataset, DataLoader
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, Spacingd, SpatialPadd, RandCropByPosNegLabeld, CenterSpatialCropd)
import random
import math
from collections import defaultdict


In [2]:
config_file = '/students/2023-2024/master/Shahpouri/DL_PET/config.json'

with open(config_file, 'r') as f:
    config = json.load(f)

log_dir = config["log_dir"]

fdg_data_dir = config["fdg_data_dir"]
fdg_output_dir = config['fdg_output_dir']

ga_output_dir = config["ga_output_dir"]
ga_data_dir = config["ga_data_dir"]

artifact_dir = config["artifacts"]
artifact_output_dir = config ["artifact_output_dir"]

artifact_repeated_dir = config["artifact_repeated_dir"]
artifacts_repeated_output_dir = config["artifacts_repeated_output_dir"]

In [6]:
from data_preparation import DataHandling 

data_handler = DataHandling(ga_data_dir, train_mode="NAC", target_mode="ADCM")

train_files = data_handler.get_data_split('train')
val_files = data_handler.get_data_split('val')
test_files = data_handler.get_data_split('test')
print(len(train_files))
print(len(val_files))
print(len(test_files))

from data_preparation import LoaderFactory
loader_factory = LoaderFactory(
    train_files=train_files,
    val_files=val_files,
    test_files=test_files,
    num_samples = 20,
    patch_size = [168, 168, 16],
    spacing = [4.07, 4.07, 3.00],
    # spacing = [1.92, 1.92, 3.27], # For fdg data
    spatial_size = (168, 168, 320)
    # spatial_size = (336, 336, 640) # for Fdg data
    )

train_loader = loader_factory.get_loader('train', batch_size=4, num_workers=2, shuffle=True)
test_loader = loader_factory.get_loader('test', batch_size=1, num_workers=2, shuffle=False)
val_loader = loader_factory.get_loader('val', batch_size=1, num_workers=2, shuffle=False)


129
33
20


Loading dataset:   0%|          | 0/129 [00:00<?, ?it/s]

Loading dataset: 100%|██████████| 129/129 [01:40<00:00,  1.29it/s]


In [4]:
import os
import torch
from model_maker import get_network

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = get_network(patch_size = [168, 168, 16], spacing = [4.07, 4.07, 3.00])

model = model.to(device)

DyUnet is set:
Kernel size:  [[3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3]]
Strides:  [[1, 1, 1], [2, 2, 2], [2, 2, 2], [2, 2, 1]]


---------
# Network Parameters Finder

In [ ]:


# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.5, 0.999))




# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [8]:
loss_function = torch.nn.MSELoss()
max_epochs = 2
val_interval = 2
best_metric = float('inf')
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
train_losses = []
val_losses = []

In [9]:
def deep_loss(outputs, target, loss_function, device, weights=None):
    """
    Compute the deep supervision loss for each output feature map.

    Parameters:
    - outputs: Tensor containing all output feature maps, including the final prediction.
    - target: The ground truth tensor.
    - loss_function: The loss function to apply.
    - device: The device on which to perform the calculations.
    - weights: A list of weights for each output's loss. Defaults to equal weighting if None.

    Returns:
    - Weighted average of the computed losses.
    """
    # Unbind the outputs along the first dimension to handle each feature map individually
    output_maps = torch.unbind(outputs, dim=1)
    
    if weights is None:
        # If no weights specified, use equal weights
        weights = [1.0 / len(output_maps)] * len(output_maps)
    elif sum(weights) != 1:
        # Normalize weights to sum to 1
        total = sum(weights)
        weights = [w / total for w in weights]

    total_loss = 0.0
    for output, weight in zip(output_maps, weights):
        # Resize target to match the output size if necessary
        resized_target = torch.nn.functional.interpolate(target, size=output.shape[2:], mode='nearest').to(device)
        # Compute loss for the current output
        loss = loss_function(output, resized_target)
        # Accumulate weighted loss
        total_loss += weight * loss

    return total_loss



In [11]:
# import functools

# def objective(trial, model, train_loader, val_loader, device, max_epochs, val_interval, loss_function, deep_loss):
#     lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    
#     optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#     criterion = torch.nn.MSELoss()

#     # Training loop
#     model.train()

#     for epoch in range(max_epochs):

#         epoch_loss = 0
#         step = 0

#         for batch_data in train_loader:
#             step += 1
#             inputs, targets = batch_data["image"].to(device), batch_data["target"].to(device)
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             if isinstance(outputs, tuple) or (outputs.dim() > targets.dim()):
#                 loss = deep_loss(outputs, targets, loss_function, device)
#             else:
#                 outputs = torch.squeeze(outputs)
#                 targets = torch.squeeze(targets, dim=1)  # Adjust for channel dimension if necessary
#                 loss = loss_function(outputs, targets)
            
#             loss.backward()
#             optimizer.step()

#             epoch_loss += loss.item()

#         epoch_loss /= step


#         # Validation logic remains largely the same
#         if (epoch + 1) % val_interval == 0:
#             model.eval()
#             val_loss = 0
#             roi_size = (168, 168, 32)
#             sw_batch_size = 16
            
#             with torch.no_grad():
#                 for val_data in val_loader:
#                     val_inputs, val_targets = val_data["image"].to(device), val_data["target"].to(device)

#                     val_outputs = sliding_window_inference(val_inputs, roi_size, sw_batch_size, model)
#                     val_loss += loss_function(val_outputs, val_targets).item()

#             val_loss /= len(val_loader)
#     return val_loss

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Use functools.partial to pass extra arguments to the objective function
# objective_with_args = functools.partial(
#     objective,
#     model=model,
#     train_loader=train_loader,
#     val_loader=val_loader,
#     device=device,
#     max_epochs=max_epochs,
#     val_interval=val_interval,
#     loss_function=loss_function,
#     deep_loss=deep_loss
# )

# study = optuna.create_study(direction='minimize')
# study.optimize(objective_with_args, n_trials=10) 

# print("Best trial:", study.best_trial.number)
# print("Best value (validation loss):", study.best_value)
# print("Best hyperparameters:", study.best_params)

---------------------

# Result for ADCM:

[I 2024-03-13 07:50:52,836] A new study created in memory with name: no-name-9bbcd5b9-2984-4e90-bb09-d75679945a16
[I 2024-03-13 07:58:07,865] Trial 0 finished with value: 0.026119201553656775 and parameters: {'lr': 0.008072853294952104}. Best is trial 0 with value: 0.026119201553656775.
[I 2024-03-13 08:05:24,497] Trial 1 finished with value: 0.024910740006495926 and parameters: {'lr': 0.05699468256166564}. Best is trial 1 with value: 0.024910740006495926.
[I 2024-03-13 08:12:36,228] Trial 2 finished with value: 0.013740474365048987 and parameters: {'lr': 0.0009126015047894743}. Best is trial 2 with value: 0.013740474365048987.
[I 2024-03-13 08:20:06,077] Trial 3 finished with value: 0.013423842097194317 and parameters: {'lr': 6.830399550473302e-05}. Best is trial 3 with value: 0.013423842097194317.
[I 2024-03-13 08:27:29,051] Trial 4 finished with value: 0.012969923698726822 and parameters: {'lr': 0.00020615854432982388}. Best is trial 4 with value: 0.012969923698726822.
[I 2024-03-13 08:35:12,140] Trial 5 finished with value: 0.012929829809924258 and parameters: {'lr': 3.557901372471996e-05}. Best is trial 5 with value: 0.012929829809924258.
[I 2024-03-13 08:43:13,100] Trial 6 finished with value: 0.012899920029346557 and parameters: {'lr': 5.8365845148990886e-05}. Best is trial 6 with value: 0.012899920029346557.
[I 2024-03-13 08:50:58,246] Trial 7 finished with value: 0.012893239296424915 and parameters: {'lr': 1.8475974753442264e-05}. Best is trial 7 with value: 0.012893239296424915.
[I 2024-03-13 08:58:42,021] Trial 8 finished with value: 0.012873913805164835 and parameters: {'lr': 5.156637879231117e-05}. Best is trial 8 with value: 0.012873913805164835.
[I 2024-03-13 09:06:27,394] Trial 9 finished with value: 0.012866821701583616 and parameters: {'lr': 1.82173317485587e-05}. Best is trial 9 with value: 0.012866821701583616.
Best trial: 9
Best value (validation loss): 0.012866821701583616
Best hyperparameters: {'lr': 1.82173317485587e-05}

In [12]:
import torch
import json
import optuna
from model_maker import get_network
from data_preparation import DataHandling, LoaderFactory

def objective(trial, data_handler):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = get_network(patch_size=[168, 168, 16], spacing=[4.07, 4.07, 3.00])
    model.to(device)

    batch_size = trial.suggest_int('batch_size', 2, 16, step=2)
    sw_batch_size = trial.suggest_int('sw_batch_size', 4, 32, step=4)
    
    loader_factory = LoaderFactory(
        train_files=data_handler.get_data_split('train'),
        val_files=data_handler.get_data_split('val'),
        test_files=data_handler.get_data_split('test'),
        patch_size=[168, 168, 16],
        spacing=[4.07, 4.07, 3.00],
        spatial_size=(168, 168, 320)
    )
    
    train_loader = loader_factory.get_loader('train', batch_size=batch_size, num_workers=2, shuffle=True)
    val_loader = loader_factory.get_loader('val', batch_size=1, num_workers=2, shuffle=False)
    
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_function = torch.nn.MSELoss()
    max_epochs = 2
    val_interval = 2

    # Training and validation logic here
    for epoch in range(max_epochs):
        model.train()
        epoch_loss = 0.0
        for batch_data in train_loader:
            inputs, targets = batch_data["image"].to(device), batch_data["target"].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, targets)
            loss.backward()
            optimizer.step()
            epochloss += loss.item()

        # Validation logic
        if (epoch + 1) % val_interval == 0:
            model.eval()
            with torch.no_grad():
                val_loss = 0
                for val_data in val_loader:
                    val_inputs, val_targets = val_data["image"].to(device), val_data["target"].to(device)
                    val_outputs = model(val_inputs)
                    val_loss += loss_function(val_outputs, val_targets).item()
            val_loss /= len(val_loader)

    return val_loss

# Loading config and setting up data handling
config_file = '/students/2023-2024/master/Shahpouri/DL_PET/config.json'
with open(config_file, 'r') as f:
    config = json.load(f)

data_handler = DataHandling(config["ga_data_dir"], train_mode="NAC", target_mode="ADCM")

# Optuna study
study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, data_handler), n_trials=10)

print("Best trial:", study.best_trial.number)
print("Best value (validation loss):", study.best_value)
print("Best hyperparameters:", study.best_params)


[I 2024-05-10 20:18:18,473] A new study created in memory with name: no-name-d99ec2da-cbb7-4b51-845a-90a5bb864b1d


DyUnet is set:
Kernel size:  [[3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3]]
Strides:  [[1, 1, 1], [2, 2, 2], [2, 2, 2], [2, 2, 1]]


Loading dataset:   5%|▍         | 6/129 [00:04<01:35,  1.29it/s]
[W 2024-05-10 20:18:23,222] Trial 0 failed with parameters: {'batch_size': 4, 'sw_batch_size': 4} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/pool.py", line 856, in next
    item = self._items.popleft()
           ^^^^^^^^^^^^^^^^^^^^^
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/homes/zshahpouri/DLP/.venv/lib/python3.11/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_3160577/866976852.py", line 68, in <lambda>
    study.optimize(lambda trial: objective(trial, data_handler), n_trials=10)
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_3160577/866976852.py", line 24, in objective
    trai

KeyboardInterrupt: 